### 토크나이저 및 모델 초기화

In [1]:
import torch
from transformers import BartForConditionalGeneration, AutoTokenizer

device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu') # gpu
tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2') # tokenizer
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2').to(device) # model

C:\Anaconda3\envs\torch-py38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


### 데이터셋 불러오기

In [2]:
import os
import re
import json
import datasets
import pandas as pd
from glob import glob
from datasets import load_dataset

**Train Data**

In [3]:
passages = []
summaries = []

# [ AI-HUB ] 요약문 및 레포트 생성 데이터 : 두 개의 폴더에 나눠져 있는 데이터를 리스트에 모아준다
## 2~3sent
for f_name in glob(os.path.join("D:/jupyter/data/요약문_및_레포트_생성/train","literature/2~3sent/*.json")) :
    with open(f_name, 'r', encoding = 'utf-8') as f :
        json_data = json.load(f)
        passages.append(json_data['Meta(Refine)']['passage'])
        summaries.append(json_data['Annotation']['summary1'])

## 20per
for f_name in glob(os.path.join("D:/jupyter/data/요약문_및_레포트_생성/train","literature/20per/*.json")) :
    with open(f_name, 'r', encoding = 'utf-8') as f :
        json_data = json.load(f)
        passages.append(json_data['Meta(Refine)']['passage'])
        summaries.append(json_data['Annotation']['summary1'])
        
## 한자 제거
passages = [re.sub("\([^\(\)]+\)", "", passage) for passage in passages]

In [4]:
# To Dataframe
train_df = pd.DataFrame([ x for  x in zip(summaries, passages)], columns=["input_texts","target_texts"])

**Valid Data**

In [5]:
passages = []
summaries = []

# [ AI-HUB ] 요약문 및 레포트 생성 데이터 : 두 개의 폴더에 나눠져 있는 데이터를 리스트에 모아준다
## 2~3sent
for f_name in glob(os.path.join("D:/jupyter/data/요약문_및_레포트_생성/valid","literature/2~3sent/*.json")) :
    with open(f_name, 'r', encoding = 'utf-8') as f :
        json_data = json.load(f)
        passages.append(json_data['Meta(Refine)']['passage'])
        summaries.append(json_data['Annotation']['summary1'])

## 20per
for f_name in glob(os.path.join("D:/jupyter/data/요약문_및_레포트_생성/valid","literature/20per/*.json")) :
    with open(f_name, 'r', encoding = 'utf-8') as f :
        json_data = json.load(f)
        passages.append(json_data['Meta(Refine)']['passage'])
        summaries.append(json_data['Annotation']['summary1'])
        
## 한자 제거
passages = [re.sub("\([^\(\)]+\)", "", passage) for passage in passages]

In [6]:
# Valid / Test 데이터 분리
valid_df = pd.DataFrame([ x for x in zip(summaries, passages)], columns = ['input_texts','target_texts'])
test_df = valid_df[600:]
valid_df = valid_df[:600]

**Dataset**

In [7]:
from datasets import Dataset

# To CSV
train_df.to_csv("train_data.csv", index = False)
valid_df.to_csv("valid_data.csv", index = False)
test_df.to_csv("test_data.csv", index = False)

# To Dataset
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

**Data 살펴보기**

In [8]:
print('[ Train ]')
print('\ninput : ', train_dataset['input_texts'][500])
print('\ntarget : ', train_dataset['target_texts'][500])

[ Train ]

input :  평목이 깊게 잠들길 기다렸던 조신은 평목의 입을 막아야겠다는 생각에 목을 매어 죽이기로 했다.

target :   "어리기는 열 다섯 살이 어려요?" 평목의 눈이 빛났다. 조신은 한 번 더 동이덩이 같은 것이 치미는 것을 삼켜버렸다. "자, 인제 늦었으니 잡시다. 내일 마누라하고도 의논해서 좋도록 하십시 다." 조신은 이렇게 말하고 자리에 누웠다. 평목도 누웠다. 조신은 잠이 들지 아니하였다. 헛코를 골면서 평목이 하는 양을 엿보았다.
평목은 잠이 드는 모양이었다. 평목이 코를 고는 것을 보고야 조신은 마음을 놓았다. 평목이 깊이 잠이 들기를 기다려서 조신은 소리 아니 나게 일어났다. "암만해도 평목의 입을 막아놓아야 할 것이다." 조신은 이렇게 생각하고 구석에 놓인 도끼를 생각하였으나 방과 몸에 피가 묻어서 형적이 남을 것을 생각하고는 목을 매어 죽이기로 하였다. 조신은 손에 맞는 끈을 생각하다가 허리띠를 끌렀다. 평목이 꿈을 꾸는지 무슨 소리를 지절거리며 돌아 누웠다. 조신은 죽온 듯이 가만히 있었다. 그러나 평목이 움직이는 것을 보고는 죽 이는 것이 무서워졌다. <사람을 죽이다니.> 하고 조신은 진저리를 쳤다. 그렇지마는 평목을 살려두고는 조신 제 몸이 온전할 수가 없었다. 평목에 게 딸을 주기는 싫었다. 딸 거울보고는 아비는 아니 닮고 어미를 닮아서 어 여뻤다. 그러한 딸을 능구렁이 같은 평목에게 준다는 것은 차마 못 할 일이 었다. 그뿐 아니다. 설사 딸을 평목에게 주더라도 그것만으로 평목이 가만 있을 것 같지 아니하였다. 필시 재물도 달라고 할 것이다. 딸을 주고 재물을 주 면 조신의 복락은 다 깨어져버리고 말 것이다.



In [9]:
print('[ Valid ]')
print('\ninput : ', valid_dataset['input_texts'][500])
print('\ntarget : ', valid_dataset['target_texts'][500])

[ Valid ]

input :  이 서방의 전처가 칼에 맞아 죽었다는 유리의 말에 어머니도 몸서리쳤다.

target :  “응! 이 서방의 전처가 아즉 죽지 않았다니…… 자, 얼굴을 들고 속 시원 하게 말을 좀 하려무나.” 유리는 그대로 얼굴을 들지 않았다. 울음을 끈치지 않으며, “네, 살았어요. 어젯밤에 저의 든 여관에 들었어요.” “너희와 한 여관에, 한 여관에.” 라고, 뇌일 뿐이러니 점점 사건이 여간 중대치 않은 데 짐작이나 선 모양이 다.
“그러면 너는 이 서방의 안해가 아니로구나. 아아, 아아, 어제 혼례를 마 치고 기뻐했더니만. 그렇다면 우는 것도 괴이찮다. …… 이 애, 이 애, 인제 울지 말고…….” “어머니는 아즉도 다 모르셔요, 어젯밤에 그 안해란 여자가 별안간에 또 죽었답니다.” 어머니는 한참 생각을 돌리는 모양이더니, “죽었어? 이상한 일도 있다. 죽었다면 세상에 웃음거리가 되지 않도록, 곧 혼례를 다시 지내고……. 그 일은 입 밖에 내지 못하게 하고.” 유리는 몸서리를 치며, “그냥 죽은 게 아녜요. 맞아 죽었어요. 칼에 맞아…….” 어머니도 몸서리를 쳤다.
“맞아 죽었어?” 그때에야 유리는 어머니의 가슴에서 떠나 교의에 몸을 던지었다.
“어머니, 어머니!” 하고는 다시 쓰러져서 진저리를 치며 운다.
“어머니는 아즉도 다 모르셔요.” 어머니는 번개같이 딸의 말뜻이 어데 있는지 깨달을 수 있었다.
“이 애, 너 그게 무슨 말이냐? 슬픔에 겨워서 네가 미쳤나 부다.” “그래요, 제가 미쳤습니다. 미쳤습니다. 어머니, 저는 고만 죽고 싶어 요.” “그렇잖다, 그럴 리는 만무하다. 의심하는 네가 그르다. 누가 무에라고 해도 이 서방은 그럴 사람이 아니다. 아니고 말고……. 그런 무서운 의심을 종작 없이…….” 유리는 또 한 번 재우쳤다.



In [10]:
print('[ Test ]')
print('\ninput : ', test_dataset['input_texts'][500])
print('\ntarget : ', test_dataset['target_texts'][500])

[ Test ]

input :  양은 균정을 모시고 자라 균정이 어느모로 뜯어보아도 부족한 데가 없는 인물로 후계왕에 가장 적임자라고 굳게 믿는다.

target :  양은 균정과 가까이 살았다. 어렸을때부터 균정을 모시며 자랐으니만치 균정의 위인을 잘 안다. 대행왕께 적왕자가 있었으면 다른 말 쓸데 없지만, 그렇지 못하면 균정이야말로 후계왕으로 가장 적임자라고 양은 굳게 믿는다. 그 인품, 인격― 천 년에 가까운 이 사직을 물려받을 중 한 자리라, 소홀히 결정하였다가는 큰일이다.
양이 잘 아는 바, 균정은 어느모로 뜯어보아도 추호 부족한 데가 없는 인 물이다. 더우기 만약 균정이 그 자리에 아니 오르면 당연한 순서로 거기 오 를 사람인 김제륭은 사람이 약하고 게다가 좀 경망하였다. 임금이 되기에는 부족하였다.
만약 균정이라는 사람이 없고, 제륭 단 혼자면 세 부득이하지만, 균정이라 는 훌륭한 적임자가 있고야, 왜 부족한 이를 위에 모시랴.
그렇게 되면 그것은 국가의 괴변이다. 이런 괴변이 생기지 않도록 사전에 방지하기 위하여, 양은, 황황히 국상 중의 서울로 달려온 것이었다.
달려와서는 그래도 좀 주저하는 균정을 등 밀다시피 해서 적판궁 으로 들여 모시었다. 일가 병사들로 숙위케 하였다.
일이 이렇게 되매, 경쟁자인 파에서도 가만 있지 않았다. 군사를 풀어 적 판궁을 둘러쌌다.
양은 여기 오산을 한 것이다. 균정을 대궐로 모시어 즉위케 하고, 국왕의 명으로 호령하면, 그 뒤는 일이 순순히 될 줄로 믿었다. 국왕의 명 령에 거역하면 이는 즉 반역이니까…. 그렇기 때문에 충분한 병력의 준비도 없이 즉위의절차부터 하였던 것이었다.



**Processing**

In [11]:
# 모델에 입력할 수 있도록 전처리를 수행하는 함수
def preprocess_function(examples):
    inputs = [doc for doc in examples["input_texts"]]
    
    # 토큰화
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target_texts"], max_length=max_target_length, truncation=True,padding='max_length')
    
    # 토큰화를 수행한 타겟 데이터를 모델 입력에 추가
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
# 입력과 타겟 문장 토큰의 최대 길이 지정
max_input_length = 64
max_target_length = 512

In [13]:
# map() function speeds up tokenization
train_data =train_dataset.map(preprocess_function, batched=True).remove_columns(["input_texts","target_texts","token_type_ids"])
valid_data =valid_dataset.map(preprocess_function, batched=True).remove_columns(["input_texts","target_texts","token_type_ids"])
test_data =test_dataset.map(preprocess_function, batched=True).remove_columns(["input_texts","target_texts","token_type_ids"])


print(train_data)
print(valid_data)
print(test_data)

Parameter 'function'=<function preprocess_function at 0x000002012F4BD798> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.01ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9600
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 600
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 600
})


### 모델 훈련 준비

**Arguments**

In [14]:
from transformers import Seq2SeqTrainingArguments

batch_size = 4

training_args = Seq2SeqTrainingArguments(
    output_dir="./kobart_bertscore_epoch-3",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    do_train=True,
    do_eval=True,
    fp16=True,
    predict_with_generate=True,
)

In [15]:
from transformers import DataCollatorForSeq2Seq, get_cosine_schedule_with_warmup

# DataCollator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

**Metric Examples**

[Huggingface : evaluate-metric](https://huggingface.co/evaluate-metric)

In [53]:
# 허깅페이스 제공 평가 지표 목록
from datasets import list_metrics

metrics_list = list_metrics()
print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, brier_score, cer, chrf, code_eval, comet, competition_math, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, matthews_correlation, mauve, mean_iou, meteor, mse, pearsonr, perplexity, poseval, precision, recall, rl_reliability, roc_auc, rouge, sacrebleu, sari, seqeval, spearmanr, squad, squad_v2, super_glue, ter, trec_eval, wer, wiki_split, xnli, xtreme_s, Felipehonorato/my_metric, GMFTBY/dailydialog_evaluate, GMFTBY/dailydialogevaluate, KevinSpaghetti/accuracyk, NikitaMartynov/spell-check-metric, NimaBoscarino/weat, Ochiroo/rouge_mn, Vertaix/vendiscore, Viona/infolm, Vlasta/pr_auc, abdusahmbzuai/aradiawer, abidlabs/mean_iou, abidlabs/mean_iou2, angelina-wang/directional_bias_amplification, cakiki/ndcg, codeparrot/apps_metric, cpllab/syntaxgym, daiyizheng/valid, erntkn/dice_coefficient, giulio98/code_eval_outputs, gnail/cosine_similarity, gorkaartola/metric_for_tp_fp_samples, hack/test_metric, idsedykh/codeble

In [2]:
# Metric usage example : BERTScore
from evaluate import load

predictions = ["hello here", "general kenobi"]
references = ["hello there", "general kenobi"]
bertscore = load("bertscore")
bert_result = bert_score.compute(predictions=predictions, references=references, lang='en')

print("[ BERTScore ]")
print(bert_result)

[ BERTScore ]
{'precision': [0.938406229019165, 1.0], 'recall': [0.938406229019165, 1.0], 'f1': [0.938406229019165, 1.0], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.11(hug_trans=4.21.2)'}


In [7]:
# Metric usage example : BLEU with NLTK
from nltk.tokenize import word_tokenize

predictions = ["hello there general kenobi", "foo bar foobar"]
references = [["hello there general kenobi"], ["foo bar foobar"]]
bleu = load("bleu")
bleu_result = bleu.compute(predictions=predictions, references=references, tokenizer=word_tokenize)

print("[ BLEU ]")
print(bleu_result)

[ BLEU ]
{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 7, 'reference_length': 7}


In [4]:
# Metric usage example : ROUGE
predictions = ["hello here", "general kenobi"]
references = ["hello there", "general kenobi"]
rouge = load("rouge")
rouge_result = rouge.compute(predictions=predictions, references=references)

print("[ Rouge ]")
print(rouge_result)

[ Rouge ]
{'rouge1': 0.75, 'rouge2': 0.5, 'rougeL': 0.75, 'rougeLsum': 0.75}


**Metric**

학습 시에는 BERTScore만을 사용한다

In [16]:
import numpy as np
from evaluate import load

metric = load("bertscore")

In [17]:
# Trainer에 전달할 평가 함수
import numpy

def compute_metrics(eval_preds) :
    preds, labels = eval_preds
    if isinstance(preds, tuple) :
        preds = preds[0]
    
    # Decoding
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) # 생성 결과 텍스트 디코딩
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # 레이블 내 -100 교체
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) # 레이블 텍스트 디코딩

     # Processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    # BERTScore 계산
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,lang="en")

    # Median 값 추출
    result = {key : np.median(value) for key, value in result.items() if type(value) == list}
    result = {k : round(v, 4) for k, v in result.items()}
    
    return result

In [18]:
# Trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = valid_data,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


In [19]:
trainer.train()

C:\Anaconda3\envs\torch-py38\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9600
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 7200


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,3.400600,3.293159,0.895900,0.827800,0.860900
2,3.274300,3.248249,0.894500,0.825200,0.858200
3,3.219300,3.236865,0.897700,0.830500,0.862900


Saving model checkpoint to ./kobart_bertscore_epoch-3\checkpoint-500
Configuration saved in ./kobart_bertscore_epoch-3\checkpoint-500\config.json
Model weights saved in ./kobart_bertscore_epoch-3\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./kobart_bertscore_epoch-3\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./kobart_bertscore_epoch-3\checkpoint-500\special_tokens_map.json
Saving model checkpoint to ./kobart_bertscore_epoch-3\checkpoint-1000
Configuration saved in ./kobart_bertscore_epoch-3\checkpoint-1000\config.json
Model weights saved in ./kobart_bertscore_epoch-3\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ./kobart_bertscore_epoch-3\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ./kobart_bertscore_epoch-3\checkpoint-1000\special_tokens_map.json
Saving model checkpoint to ./kobart_bertscore_epoch-3\checkpoint-1500
Configuration saved in ./kobart_bertscore_epoch-3\checkpoint-1500\config.json
Model wei

Saving model checkpoint to ./kobart_bertscore_epoch-3\checkpoint-4500
Configuration saved in ./kobart_bertscore_epoch-3\checkpoint-4500\config.json
Model weights saved in ./kobart_bertscore_epoch-3\checkpoint-4500\pytorch_model.bin
tokenizer config file saved in ./kobart_bertscore_epoch-3\checkpoint-4500\tokenizer_config.json
Special tokens file saved in ./kobart_bertscore_epoch-3\checkpoint-4500\special_tokens_map.json
Deleting older checkpoint [kobart_bertscore_epoch-3\checkpoint-3000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 600
  Batch size = 4
Saving model checkpoint to ./kobart_bertscore_epoch-3\checkpoint-5000
Configuration saved in ./kobart_bertscore_epoch-3\checkpoint-5000\config.json
Model weights saved in ./kobart_bertscore_epoch-3\checkpoint-5000\pytorch_model.bin
tokenizer config file saved in ./kobart_bertscore_epoch-3\checkpoint-5000\tokenizer_config.json
Special tokens file saved in ./kobart_bertscore_epoch-3\checkpoint-5000\special_t

TrainOutput(global_step=7200, training_loss=3.3520824347601996, metrics={'train_runtime': 1647.227, 'train_samples_per_second': 17.484, 'train_steps_per_second': 4.371, 'total_flos': 1097525624832000.0, 'train_loss': 3.3520824347601996, 'epoch': 3.0})

### 테스트 데이터 평가

**테스트 데이터 불러오기**

In [1]:
import pandas as pd

test_df = pd.read_csv('./test_data.csv')
test_inputs = test_df['input_texts'].tolist()
test_outputs = test_df['target_texts'].tolist()

**모델 불러오기**

In [22]:
from transformers import BartForConditionalGeneration, AutoTokenizer

# KoBART
test_tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')
test_model = BartForConditionalGeneration.from_pretrained('./kobart_bertscore_epoch-3/checkpoint-7000')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gogamza/kobart-base-v2/resolve/main/config.json from cache at C:\Users\PC/.cache\huggingface\transformers\54a37e9385f90886428b084042f151c1a699203416d41765d94aac4cddb5fd5c.d098ef3866c1da94bdfaa5c1f24ecb7c5c16b37423b79263fbd3668d2ae61f91
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  

**문장 생성하기**

In [26]:
from tqdm import tqdm

test_outputs = []
for i, test_input in enumerate(tqdm(test_inputs)) :
    input_ids = test_tokenizer.encode(test_input, return_tensors='pt')
    gen_ids = test_model.generate(input_ids,
                                 do_sample = True,
                                 max_length = 512,
                                 min_length = 64,
                                 repetition_penalty = 1.5,
                                 no_repeat_ngram_size = 3,
                                 temperature = 0.9,
                                 top_k = 50,
                                 top_p = 1.0)
    generated = test_tokenizer.decode(gen_ids[0])
    test_outputs.append(generated)

100%|██████████████████████████████████████████████████████████████████████████████| 600/600 [2:32:57<00:00, 15.30s/it]


In [30]:
# To Dataframe
generated_df = pd.DataFrame([ x for  x in zip(test_inputs, test_outputs)], columns=["test_input","test_output"])

In [31]:
generated_df.to_csv("generated.csv", index = False)

In [47]:
index = 100
print("\ninput : ", generated_df.iloc[index]['test_input'])
print("\noutput : ", generated_df.iloc[index]['test_output'])


input :  찬도는 괴상한 물건에 의해 정신이 어지러워져 온 힘을 다하려 수습해 보려 했으나 정신을 차리지 못하고 약 10분이 자나서야 정신을 차렸다.

output :  </s> 찬도는 이즈음에 괴상한 물건에 의하여 정신이 어지러워졌으나 이를 수습해 보려니 별안간 정신을 차릴 수가 없었다.
얼마 뒤 기침 소리가 났을 때였는고. “일어나서 불을 지펴 주시오?” 고 누가 물었다. 그리고 난데없는 외침을 일삼아 선뜻 좇아가지 않았다. 간혹 무슨 이야기를 하여도 알아들으려 하지 않는다. 그러나 그 모든 이야기가 다 거짓이라고 얼른 믿을 수는 없는 노릇이었다.
찬도의 가슴은 더 끓기 시작하였다. 그는 정신으로 다시 자시질 않으니 어찌 할 수 없 기도 했다.
남의 앞길을 막으려면 어떻게 해서 해야할는지.......
그렇게 말을 하고자 하면 안될 것이나, 어느 틈엔가 발작이 일어나서 그의 귀를 누르고 서 있는 사람이 있었 다. 또 한 사람에게는 아주 무서운 불길이 솟았다. 그때, 남한테 당한 듯이 몹시 혼잣말같이 들리는 소리를 들었으나, 아랫목에서부터 부르는 호령을 들을 때, 그것은 너무도 큰 소리로 부르짖 는 소리였다. 아무리 소리에 놀라도, 목에서는 피가 줄줄 흐르다가 조금 걷잡힐 까닭 없이 토막을 내 버렸다. 그리하여 온 힘을 다해 겨우 남은 몸을 밖으로 끌고 나가서, ‘인제는 그만하면 됐겠다!’ 생각하였다.
3 가느다란 돌로 만든 두터운 벽장 같은 방에 형체가 없었다. 그래도 그대로 서서만 있지 않았다면 저절로 숨을 쉬기도 하였을까? 대체 누구에게 이렇게 아픈 곳이 있다는 걸 깨달으면 지금쯤 어렸을 때의 기억이 머릿속에 번득번뜩 떠오르는 것이다. 그럴 때는 벌써 몇 해 전이다. 동정의 눈으로 그를 쳐다보던 날 밤이었다. 어떤 날, 젊은 친구 집앞에서 자고 있던 조무래기와 놀이를 하던 중에 갑자기 웬 요란한 음성이 들려왔습니다.
<pad><pad><pad><unused56><unused70><pad><unused21><unused68><unused37>어졌다.

**평가 점수 계산**

#### [ BERTScore ]

- precision: The precision for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.


- recall: The recall for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.


- f1: The F1 score for each sentence from the predictions + references lists, which ranges from 0.0 to 1.0.

In [65]:
# BERTScore
from evaluate import load

bert_score = load("bertscore")
bert_result = bert_score.compute(predictions=test_outputs, references=list(test_df['target_texts']), lang='ko')

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at C:\Users\PC/.cache\huggingface\transformers\6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type"

In [66]:
bert_result = {key : np.median(value) for key, value in bert_result.items() if type(value) == list}
print("[ BERTScore ]")
print(bert_result)

[ BERTScore ]
{'precision': 0.6934953927993774, 'recall': 0.7104206383228302, 'f1': 0.701668381690979}


#### [ BLEU ]

- BLEU’s output is always a number between 0 and 1.

  This value indicates how similar the candidate text is to the reference texts,
    
  with values closer to 1 representing more similar texts

In [74]:
from eunjeon import Mecab

mecab = Mecab()

In [89]:
# BLEU Score (w. Mecab)
from eunjeon import Mecab
from datasets import load_metric

mecab = Mecab()
bleu = load_metric("bleu")

tokenized_test_outputs = [[mecab.morphs(sen)] for sen in list(test_df['target_texts'])]
tokenized_generated = [mecab.morphs(sen) for sen in test_outputs]
bleu_result = bleu.compute(predictions=tokenized_generated, references=tokenized_test_outputs)

In [90]:
print("[ BLEU ]")
print(bleu_result)

[ BLEU ]
{'bleu': 0.036876324441368256, 'precisions': [0.24055568103892086, 0.06035250278607999, 0.018990969600169716, 0.006707064270586337], 'brevity_penalty': 1.0, 'length_ratio': 1.913024353025443, 'translation_length': 491433, 'reference_length': 256888}


#### [[ ROUGE ]](https://dacon.io/competitions/open/235671/overview/rules)

- DACON의 한국어 문서 추출요약 AI 경진대회에서 사용된 ROUGE 스코어 산식 코드를 사용하였다


- 해당 코드는 py-rouge 소스코드를 한글에 맞게 수정하였다

In [92]:
import os
import re
import platform
import itertools
import collections
import pkg_resources  # pip install py-rouge
from io import open


if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
    from konlpy.tag import Mecab




class Rouge:
    DEFAULT_METRICS = {"rouge-n"}
    DEFAULT_N = 1
    STATS = ["f", "p", "r"]
    AVAILABLE_METRICS = {"rouge-n", "rouge-l", "rouge-w"}
    AVAILABLE_LENGTH_LIMIT_TYPES = {"words", "bytes"}
    REMOVE_CHAR_PATTERN = re.compile("[^A-Za-z0-9가-힣]")


    def __init__(
        self,
        metrics=None,
        max_n=None,
        limit_length=True,
        length_limit=1000,
        length_limit_type="words",
        apply_avg=True,
        apply_best=False,
        use_tokenizer=True,
        alpha=0.5,
        weight_factor=1.0,
    ):
        self.metrics = metrics[:] if metrics is not None else Rouge.DEFAULT_METRICS
        for m in self.metrics:
            if m not in Rouge.AVAILABLE_METRICS:
                raise ValueError("Unknown metric '{}'".format(m))


        self.max_n = max_n if "rouge-n" in self.metrics else None
        # Add all rouge-n metrics
        if self.max_n is not None:
            index_rouge_n = self.metrics.index("rouge-n")
            del self.metrics[index_rouge_n]
            self.metrics += ["rouge-{}".format(n) for n in range(1, self.max_n + 1)]
        self.metrics = set(self.metrics)


        self.limit_length = limit_length
        if self.limit_length:
            if length_limit_type not in Rouge.AVAILABLE_LENGTH_LIMIT_TYPES:
                raise ValueError("Unknown length_limit_type '{}'".format(length_limit_type))


        self.length_limit = length_limit
        if self.length_limit == 0:
            self.limit_length = False
        self.length_limit_type = length_limit_type


        self.use_tokenizer = use_tokenizer
        if use_tokenizer:
            self.tokenizer = Mecab()


        self.apply_avg = apply_avg
        self.apply_best = apply_best
        self.alpha = alpha
        self.weight_factor = weight_factor
        if self.weight_factor <= 0:
            raise ValueError("ROUGE-W weight factor must greater than 0.")


    def tokenize_text(self, text):
        if self.use_tokenizer:
            return self.tokenizer.morphs(text)
        else:
            return text


    @staticmethod
    def split_into_sentences(text):
        return text.split("\n")


    @staticmethod
    def _get_ngrams(n, text):
        ngram_set = collections.defaultdict(int)
        max_index_ngram_start = len(text) - n
        for i in range(max_index_ngram_start + 1):
            ngram_set[tuple(text[i : i + n])] += 1
        return ngram_set


    @staticmethod
    def _split_into_words(sentences):
        return list(itertools.chain(*[_.split() for _ in sentences]))


    @staticmethod
    def _get_word_ngrams_and_length(n, sentences):
        assert len(sentences) > 0
        assert n > 0


        tokens = Rouge._split_into_words(sentences)
        return Rouge._get_ngrams(n, tokens), tokens, len(tokens) - (n - 1)


    @staticmethod
    def _get_unigrams(sentences):
        assert len(sentences) > 0


        tokens = Rouge._split_into_words(sentences)
        unigram_set = collections.defaultdict(int)
        for token in tokens:
            unigram_set[token] += 1
        return unigram_set, len(tokens)


    @staticmethod
    def _compute_p_r_f_score(
        evaluated_count,
        reference_count,
        overlapping_count,
        alpha=0.5,
        weight_factor=1.0,
    ):
        precision = 0.0 if evaluated_count == 0 else overlapping_count / float(evaluated_count)
        if weight_factor != 1.0:
            precision = precision ** (1.0 / weight_factor)
        recall = 0.0 if reference_count == 0 else overlapping_count / float(reference_count)
        if weight_factor != 1.0:
            recall = recall ** (1.0 / weight_factor)
        f1_score = Rouge._compute_f_score(precision, recall, alpha)
        return {"f": f1_score, "p": precision, "r": recall}


    @staticmethod
    def _compute_f_score(precision, recall, alpha=0.5):
        return (
            0.0
            if (recall == 0.0 or precision == 0.0)
            else precision * recall / ((1 - alpha) * precision + alpha * recall)
        )


    @staticmethod
    def _compute_ngrams(evaluated_sentences, reference_sentences, n):
        if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
            raise ValueError("Collections must contain at least 1 sentence.")


        evaluated_ngrams, _, evaluated_count = Rouge._get_word_ngrams_and_length(
            n, evaluated_sentences
        )
        reference_ngrams, _, reference_count = Rouge._get_word_ngrams_and_length(
            n, reference_sentences
        )


        # Gets the overlapping ngrams between evaluated and reference
        overlapping_ngrams = set(evaluated_ngrams.keys()).intersection(set(reference_ngrams.keys()))
        overlapping_count = 0
        for ngram in overlapping_ngrams:
            overlapping_count += min(evaluated_ngrams[ngram], reference_ngrams[ngram])


        return evaluated_count, reference_count, overlapping_count


    @staticmethod
    def _compute_ngrams_lcs(evaluated_sentences, reference_sentences, weight_factor=1.0):
        def _lcs(x, y):
            m = len(x)
            n = len(y)
            vals = collections.defaultdict(int)
            dirs = collections.defaultdict(int)


            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if x[i - 1] == y[j - 1]:
                        vals[i, j] = vals[i - 1, j - 1] + 1
                        dirs[i, j] = "|"
                    elif vals[i - 1, j] >= vals[i, j - 1]:
                        vals[i, j] = vals[i - 1, j]
                        dirs[i, j] = "^"
                    else:
                        vals[i, j] = vals[i, j - 1]
                        dirs[i, j] = "<"


            return vals, dirs


        def _wlcs(x, y, weight_factor):
            m = len(x)
            n = len(y)
            vals = collections.defaultdict(float)
            dirs = collections.defaultdict(int)
            lengths = collections.defaultdict(int)


            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if x[i - 1] == y[j - 1]:
                        length_tmp = lengths[i - 1, j - 1]
                        vals[i, j] = (
                            vals[i - 1, j - 1]
                            + (length_tmp + 1) ** weight_factor
                            - length_tmp ** weight_factor
                        )
                        dirs[i, j] = "|"
                        lengths[i, j] = length_tmp + 1
                    elif vals[i - 1, j] >= vals[i, j - 1]:
                        vals[i, j] = vals[i - 1, j]
                        dirs[i, j] = "^"
                        lengths[i, j] = 0
                    else:
                        vals[i, j] = vals[i, j - 1]
                        dirs[i, j] = "<"
                        lengths[i, j] = 0


            return vals, dirs


        def _mark_lcs(mask, dirs, m, n):
            while m != 0 and n != 0:
                if dirs[m, n] == "|":
                    m -= 1
                    n -= 1
                    mask[m] = 1
                elif dirs[m, n] == "^":
                    m -= 1
                elif dirs[m, n] == "<":
                    n -= 1
                else:
                    raise UnboundLocalError("Illegal move")


            return mask


        if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
            raise ValueError("Collections must contain at least 1 sentence.")


        evaluated_unigrams_dict, evaluated_count = Rouge._get_unigrams(evaluated_sentences)
        reference_unigrams_dict, reference_count = Rouge._get_unigrams(reference_sentences)


        # Has to use weight factor for WLCS
        use_WLCS = weight_factor != 1.0
        if use_WLCS:
            evaluated_count = evaluated_count ** weight_factor
            reference_count = 0


        overlapping_count = 0.0
        for reference_sentence in reference_sentences:
            reference_sentence_tokens = reference_sentence.split()
            if use_WLCS:
                reference_count += len(reference_sentence_tokens) ** weight_factor
            hit_mask = [0 for _ in range(len(reference_sentence_tokens))]


            for evaluated_sentence in evaluated_sentences:
                evaluated_sentence_tokens = evaluated_sentence.split()


                if use_WLCS:
                    _, lcs_dirs = _wlcs(
                        reference_sentence_tokens,
                        evaluated_sentence_tokens,
                        weight_factor,
                    )
                else:
                    _, lcs_dirs = _lcs(reference_sentence_tokens, evaluated_sentence_tokens)
                _mark_lcs(
                    hit_mask,
                    lcs_dirs,
                    len(reference_sentence_tokens),
                    len(evaluated_sentence_tokens),
                )


            overlapping_count_length = 0
            for ref_token_id, val in enumerate(hit_mask):
                if val == 1:
                    token = reference_sentence_tokens[ref_token_id]
                    if evaluated_unigrams_dict[token] > 0 and reference_unigrams_dict[token] > 0:
                        evaluated_unigrams_dict[token] -= 1
                        reference_unigrams_dict[ref_token_id] -= 1


                        if use_WLCS:
                            overlapping_count_length += 1
                            if (
                                ref_token_id + 1 < len(hit_mask) and hit_mask[ref_token_id + 1] == 0
                            ) or ref_token_id + 1 == len(hit_mask):
                                overlapping_count += overlapping_count_length ** weight_factor
                                overlapping_count_length = 0
                        else:
                            overlapping_count += 1


        if use_WLCS:
            reference_count = reference_count ** weight_factor


        return evaluated_count, reference_count, overlapping_count


    def get_scores(self, hypothesis, references):
        if isinstance(hypothesis, str):
            hypothesis, references = [hypothesis], [references]


        if type(hypothesis) != type(references):
            raise ValueError("'hyps' and 'refs' are not of the same type")


        if len(hypothesis) != len(references):
            raise ValueError("'hyps' and 'refs' do not have the same length")
        scores = {}
        has_rouge_n_metric = (
            len([metric for metric in self.metrics if metric.split("-")[-1].isdigit()]) > 0
        )
        if has_rouge_n_metric:
            scores.update(self._get_scores_rouge_n(hypothesis, references))
            # scores = {**scores, **self._get_scores_rouge_n(hypothesis, references)}


        has_rouge_l_metric = (
            len([metric for metric in self.metrics if metric.split("-")[-1].lower() == "l"]) > 0
        )
        if has_rouge_l_metric:
            scores.update(self._get_scores_rouge_l_or_w(hypothesis, references, False))
            # scores = {**scores, **self._get_scores_rouge_l_or_w(hypothesis, references, False)}


        has_rouge_w_metric = (
            len([metric for metric in self.metrics if metric.split("-")[-1].lower() == "w"]) > 0
        )
        if has_rouge_w_metric:
            scores.update(self._get_scores_rouge_l_or_w(hypothesis, references, True))
            # scores = {**scores, **self._get_scores_rouge_l_or_w(hypothesis, references, True)}


        return scores


    def _get_scores_rouge_n(self, all_hypothesis, all_references):
        metrics = [metric for metric in self.metrics if metric.split("-")[-1].isdigit()]


        if self.apply_avg or self.apply_best:
            scores = {metric: {stat: 0.0 for stat in Rouge.STATS} for metric in metrics}
        else:
            scores = {
                metric: [{stat: [] for stat in Rouge.STATS} for _ in range(len(all_hypothesis))]
                for metric in metrics
            }


        for sample_id, (hypothesis, references) in enumerate(zip(all_hypothesis, all_references)):
            assert isinstance(hypothesis, str)
            has_multiple_references = False
            if isinstance(references, list):
                has_multiple_references = len(references) > 1
                if not has_multiple_references:
                    references = references[0]


            # Prepare hypothesis and reference(s)
            hypothesis = self._preprocess_summary_as_a_whole(hypothesis)
            references = (
                [self._preprocess_summary_as_a_whole(reference) for reference in references]
                if has_multiple_references
                else [self._preprocess_summary_as_a_whole(references)]
            )


            # Compute scores
            for metric in metrics:
                suffix = metric.split("-")[-1]
                n = int(suffix)


                # Aggregate
                if self.apply_avg:
                    # average model
                    total_hypothesis_ngrams_count = 0
                    total_reference_ngrams_count = 0
                    total_ngrams_overlapping_count = 0


                    for reference in references:
                        (
                            hypothesis_count,
                            reference_count,
                            overlapping_ngrams,
                        ) = Rouge._compute_ngrams(hypothesis, reference, n)
                        total_hypothesis_ngrams_count += hypothesis_count
                        total_reference_ngrams_count += reference_count
                        total_ngrams_overlapping_count += overlapping_ngrams


                    score = Rouge._compute_p_r_f_score(
                        total_hypothesis_ngrams_count,
                        total_reference_ngrams_count,
                        total_ngrams_overlapping_count,
                        self.alpha,
                    )


                    for stat in Rouge.STATS:
                        scores[metric][stat] += score[stat]
                else:
                    # Best model
                    if self.apply_best:
                        best_current_score = None
                        for reference in references:
                            (
                                hypothesis_count,
                                reference_count,
                                overlapping_ngrams,
                            ) = Rouge._compute_ngrams(hypothesis, reference, n)
                            score = Rouge._compute_p_r_f_score(
                                hypothesis_count,
                                reference_count,
                                overlapping_ngrams,
                                self.alpha,
                            )
                            if best_current_score is None or score["r"] > best_current_score["r"]:
                                best_current_score = score


                        for stat in Rouge.STATS:
                            scores[metric][stat] += best_current_score[stat]
                    # Keep all
                    else:
                        for reference in references:
                            (
                                hypothesis_count,
                                reference_count,
                                overlapping_ngrams,
                            ) = Rouge._compute_ngrams(hypothesis, reference, n)
                            score = Rouge._compute_p_r_f_score(
                                hypothesis_count,
                                reference_count,
                                overlapping_ngrams,
                                self.alpha,
                            )
                            for stat in Rouge.STATS:
                                scores[metric][sample_id][stat].append(score[stat])


        # Compute final score with the average or the the max
        if (self.apply_avg or self.apply_best) and len(all_hypothesis) > 1:
            for metric in metrics:
                for stat in Rouge.STATS:
                    scores[metric][stat] /= len(all_hypothesis)


        return scores


    def _get_scores_rouge_l_or_w(self, all_hypothesis, all_references, use_w=False):
        metric = "rouge-w" if use_w else "rouge-l"
        if self.apply_avg or self.apply_best:
            scores = {metric: {stat: 0.0 for stat in Rouge.STATS}}
        else:
            scores = {
                metric: [{stat: [] for stat in Rouge.STATS} for _ in range(len(all_hypothesis))]
            }


        for sample_id, (hypothesis_sentences, references_sentences) in enumerate(
            zip(all_hypothesis, all_references)
        ):
            assert isinstance(hypothesis_sentences, str)
            has_multiple_references = False
            if isinstance(references_sentences, list):
                has_multiple_references = len(references_sentences) > 1
                if not has_multiple_references:
                    references_sentences = references_sentences[0]


            # Prepare hypothesis and reference(s)
            hypothesis_sentences = self._preprocess_summary_per_sentence(hypothesis_sentences)
            references_sentences = (
                [
                    self._preprocess_summary_per_sentence(reference)
                    for reference in references_sentences
                ]
                if has_multiple_references
                else [self._preprocess_summary_per_sentence(references_sentences)]
            )


            # Compute scores
            # Aggregate
            if self.apply_avg:
                # average model
                total_hypothesis_ngrams_count = 0
                total_reference_ngrams_count = 0
                total_ngrams_overlapping_count = 0


                for reference_sentences in references_sentences:
                    (
                        hypothesis_count,
                        reference_count,
                        overlapping_ngrams,
                    ) = Rouge._compute_ngrams_lcs(
                        hypothesis_sentences,
                        reference_sentences,
                        self.weight_factor if use_w else 1.0,
                    )
                    total_hypothesis_ngrams_count += hypothesis_count
                    total_reference_ngrams_count += reference_count
                    total_ngrams_overlapping_count += overlapping_ngrams


                score = Rouge._compute_p_r_f_score(
                    total_hypothesis_ngrams_count,
                    total_reference_ngrams_count,
                    total_ngrams_overlapping_count,
                    self.alpha,
                    self.weight_factor if use_w else 1.0,
                )
                for stat in Rouge.STATS:
                    scores[metric][stat] += score[stat]
            else:
                # Best model
                if self.apply_best:
                    best_current_score = None
                    best_current_score_wlcs = None
                    for reference_sentences in references_sentences:
                        (
                            hypothesis_count,
                            reference_count,
                            overlapping_ngrams,
                        ) = Rouge._compute_ngrams_lcs(
                            hypothesis_sentences,
                            reference_sentences,
                            self.weight_factor if use_w else 1.0,
                        )
                        score = Rouge._compute_p_r_f_score(
                            total_hypothesis_ngrams_count,
                            total_reference_ngrams_count,
                            total_ngrams_overlapping_count,
                            self.alpha,
                            self.weight_factor if use_w else 1.0,
                        )


                        if use_w:
                            reference_count_for_score = reference_count ** (
                                1.0 / self.weight_factor
                            )
                            overlapping_ngrams_for_score = overlapping_ngrams
                            score_wlcs = (
                                overlapping_ngrams_for_score / reference_count_for_score
                            ) ** (1.0 / self.weight_factor)


                            if (
                                best_current_score_wlcs is None
                                or score_wlcs > best_current_score_wlcs
                            ):
                                best_current_score = score
                                best_current_score_wlcs = score_wlcs
                        else:
                            if best_current_score is None or score["r"] > best_current_score["r"]:
                                best_current_score = score


                    for stat in Rouge.STATS:
                        scores[metric][stat] += best_current_score[stat]
                # Keep all
                else:
                    for reference_sentences in references_sentences:
                        (
                            hypothesis_count,
                            reference_count,
                            overlapping_ngrams,
                        ) = Rouge._compute_ngrams_lcs(
                            hypothesis_sentences,
                            reference_sentences,
                            self.weight_factor if use_w else 1.0,
                        )
                        score = Rouge._compute_p_r_f_score(
                            hypothesis_count,
                            reference_count,
                            overlapping_ngrams,
                            self.alpha,
                            self.weight_factor,
                        )


                        for stat in Rouge.STATS:
                            scores[metric][sample_id][stat].append(score[stat])


        # Compute final score with the average or the the max
        if (self.apply_avg or self.apply_best) and len(all_hypothesis) > 1:
            for stat in Rouge.STATS:
                scores[metric][stat] /= len(all_hypothesis)


        return scores


    def _preprocess_summary_as_a_whole(self, summary):
        sentences = Rouge.split_into_sentences(summary)


        # Truncate
        if self.limit_length:
            # By words
            if self.length_limit_type == "words":
                summary = " ".join(sentences)
                all_tokens = summary.split()  # Counting as in the perls script
                summary = " ".join(all_tokens[: self.length_limit])


            # By bytes
            elif self.length_limit_type == "bytes":
                summary = ""
                current_len = 0
                for sentence in sentences:
                    sentence = sentence.strip()
                    sentence_len = len(sentence)


                    if current_len + sentence_len < self.length_limit:
                        if current_len != 0:
                            summary += " "
                        summary += sentence
                        current_len += sentence_len
                    else:
                        if current_len > 0:
                            summary += " "
                        summary += sentence[: self.length_limit - current_len]
                        break
        else:
            summary = " ".join(sentences)


        summary = Rouge.REMOVE_CHAR_PATTERN.sub(" ", summary.lower()).strip()


        tokens = self.tokenize_text(Rouge.REMOVE_CHAR_PATTERN.sub(" ", summary))
        preprocessed_summary = [" ".join(tokens)]


        return preprocessed_summary


    def _preprocess_summary_per_sentence(self, summary):
        sentences = Rouge.split_into_sentences(summary)


        # Truncate
        if self.limit_length:
            final_sentences = []
            current_len = 0
            # By words
            if self.length_limit_type == "words":
                for sentence in sentences:
                    tokens = sentence.strip().split()
                    tokens_len = len(tokens)
                    if current_len + tokens_len < self.length_limit:
                        sentence = " ".join(tokens)
                        final_sentences.append(sentence)
                        current_len += tokens_len
                    else:
                        sentence = " ".join(tokens[: self.length_limit - current_len])
                        final_sentences.append(sentence)
                        break
            # By bytes
            elif self.length_limit_type == "bytes":
                for sentence in sentences:
                    sentence = sentence.strip()
                    sentence_len = len(sentence)
                    if current_len + sentence_len < self.length_limit:
                        final_sentences.append(sentence)
                        current_len += sentence_len
                    else:
                        sentence = sentence[: self.length_limit - current_len]
                        final_sentences.append(sentence)
                        break
            sentences = final_sentences


        final_sentences = []
        for sentence in sentences:
            sentence = Rouge.REMOVE_CHAR_PATTERN.sub(" ", sentence.lower()).strip()


            tokens = self.tokenize_text(Rouge.REMOVE_CHAR_PATTERN.sub(" ", sentence))


            sentence = " ".join(tokens)


            final_sentences.append(sentence)


        return final_sentences

In [94]:
import argparse
import pandas as pd
from tqdm import tqdm

class RougeScorer:
    def __init__(self):
        self.rouge_evaluator = Rouge(
            metrics=["rouge-n", "rouge-l"],
            max_n=2,
            limit_length=True,
            length_limit=1000,
            length_limit_type="words",
            use_tokenizer=True,
            apply_avg=True,
            apply_best=False,
            alpha=0.5,  # Default F1_score
            weight_factor=1.2,
        )

    def compute_rouge(self, ref_df, hyp_df):
        #ref_df = pd.read_csv(ref_path)
        #hyp_df = pd.read_csv(hyp_path)
        hyp_df.iloc[:,1] = hyp_df.iloc[:,1].fillna(' ')
        ids = ref_df['id']
        hyp_df = hyp_df[hyp_df['id'].isin(ids)]
        hyp_df.index = ref_df.index

        ref_df = ref_df.sort_values(by=["id"])
        hyp_df = hyp_df.sort_values(by=["id"])
        ref_df["id"] = ref_df["id"].astype(int)
        hyp_df["id"] = hyp_df["id"].astype(int)

        hyps = [tuple(row) for row in hyp_df.values]
        refs = [tuple(row) for row in ref_df.values]

        reference_summaries = []
        generated_summaries = []

        for ref_tp, hyp_tp in zip(refs, hyps):
            ref, ref_id = ref_tp
            hyp, hyp_id = hyp_tp

            assert ref_id == hyp_id

            reference_summaries.append(ref)
            generated_summaries.append(hyp)

        scores = self.rouge_evaluator.get_scores(generated_summaries, reference_summaries)
        str_scores = self.format_rouge_scores(scores)
        #self.save_rouge_scores(str_scores)
        return str_scores

    def save_rouge_scores(self, str_scores):
        with open("rouge_scores.txt", "w") as output:
            output.write(str_scores)

    def format_rouge_scores(self, scores):
    	return "{:.3f},{:.3f},{:.3f}".format(
            scores["rouge-1"]["f"],
            scores["rouge-2"]["f"],
            scores["rouge-l"]["f"],
        )

In [102]:
# ROUGE
rouge = RougeScorer()

rouge_target = pd.DataFrame(list(test_df['target_texts']), columns=['summary'])
rouge_target['id'] = rouge_target.index
rouge_generated = pd.DataFrame(test_outputs, columns=['summary'])
rouge_generated['id'] = rouge_generated.index

rouge_result = rouge.compute_rouge(rouge_target,rouge_generated)

In [104]:
print("[ Rouge ]")
print("Rouge-1, Rouge-2, Rouge-L : ", rouge_result)

[ Rouge ]
Rouge-1, Rouge-2, Rouge-L :  0.339,0.072,0.194


### 인퍼런스

In [43]:
text = '하얀 케이크를 자르고 있는 여자가 있다.'
input_ids = tokenizer.encode(text, return_tensors='pt').cuda()
gen_ids = model.generate(input_ids,
                         do_sample=True,
                         max_length=256,
                         min_length=16,
                         repetition_penalty=1.5,
                         no_repeat_ngram_size=3,
                         temperature=0.9,
                         top_k=50,
                         top_p=0.92)
generated = tokenizer.decode(gen_ids[0])
print('original : ', text)
print('generated : ', generated)

original :  하얀 케이크를 자르고 있는 여자가 있다.
generated :  </s> “누님!” 하고 여자는 소리쳤다. 그의 옆에 서 있던 여자가 벌떡 일어나며 두 손을 번쩍 들고 하얀 케이크를 자르고 있었다. 그 여자의 얼굴에는 이 런 일이 생 각이 났었다. 흰 옷도 입으신 듯싶었 다. “그리고 무슨 장난일까? 나는 퍽 오래전부터 그런 생각을 해 왔어요. 요즈음 밤마다 새까만 옷을 입고 다니는 것 같습니다. 어째서 그러우? 왜 그럴 때에 날쌔게 안 나타났을까. 내가 아무리 못 본 체해도 알 수 있나요. 그리고 또 어떤 방에서든지 나 같은 놈은 언제나 나와 같이 놀다가 가겠어요..... 그러나.......
저를 데리고 있는 사람이 누구냐? 이런 질문과 저편을 보고 웃으면서 물으니까, 우리 모두 참말 이상하리만큼 마음이 가라앉아 있답니다. 저는 물론 당신이 얼마나 똑똑하신 분인가요? 제가 만일 당신의 꿈을 이루지 못하면 당신에게 무한한 행복을 줄 것은 없습니다마는...... 선생님께서 그리시는 것을 꼭 한 가지 들어 주십시요 그래서 아유. 아까는 아무튼 잘됐어요, 그러니까 제발 저를 좀 맡아 보구려 이렇게 말하셨더니, 그럼 당신은 정말 그렇게 하세요 하셨다 고 말씀하시더라고 하시던 것입니다. 그런데 오늘 와서는 내 생각으로는 나도 모르게 영감을</s>
